In [1]:
import os 
import json
import pandas as pd 
import numpy as np
import traceback
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
from langchain.chat_models import ChatOpenAI

In [4]:
llm = ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"], model_name="gpt-3.5-turbo",temperature=0.5)

d:\LangChain\MCQ\venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [6]:
RESPONSE_JSON = {
    "1": {
        "mcq" : "multiple choice question",
        "options" : {
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq" : "multiple choice question",
        "options" : {
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct": "correct answer",        
    },
    "3": {
        "mcq" : "multiple choice question",
        "options" : {
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct": "correct answer", 
    },
}

In [7]:
TEMPLATE = """
Text:{text}
You are an expert MCQ Maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} students\
    in {tone} tone.Make sure the questions are not repeated and check all the questions to be conforming as well.The questions should be well made\
    Make sure to format  your response like RESPONSE_JSON below and use it as a guide.\
    Ensure to make {number} MCQs
    ### Response_JSON
    {response_json}
"""

In [8]:
Quiz_prompt =  PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [9]:
quiz_chain = LLMChain(llm=llm,prompt=Quiz_prompt,output_key="quiz",verbose=True)

In [10]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Question for {subject} students.You need to evaluate the Complexity of the question \
    and give a complete analysis of the quiz. Only use at max 50 words for complexity. If the quiz is not at par with the cognitive and analytical abilities\
    of the students , update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities.
    Quiz_MCQs : 
    {quiz}
        
    Check from an expert english writer of the above quiz.
"""

In [11]:
Quiz_evaluation_prompt = PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE2)

In [12]:
Quiz_evaluation_chain = LLMChain(llm=llm,prompt=Quiz_evaluation_prompt,output_key='review',verbose=True)

In [13]:
generation_chain = SequentialChain(chains=[quiz_chain,Quiz_evaluation_chain],input_variables=["text","number","subject","tone","response_json"],
                                   output_variables=["quiz","review"],verbose=True)

In [14]:
file_path = "D:\LangChain\MCQ\data.txt"

In [15]:
file_path

'D:\\LangChain\\MCQ\\data.txt'

In [16]:
with open(file_path,'r') as file:
    Text =  file.read()

In [17]:
print(Text)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive systems contributed

In [18]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [19]:
Number = 4
SUBJECT = "Machine Learning"
TONE = "simple"

In [20]:
with get_openai_callback() as cb:
    response = generation_chain(
        {
            "text":Text,
            "number":Number,
            "subject":SUBJECT,
            "tone":TONE,
            "response_json":json.dumps(RESPONSE_JSON)   
        }
    )

d:\LangChain\MCQ\venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used 

In [21]:
print(f"Total Tokens : {cb.total_tokens}")
print(f"Prompt Tokens : {cb.prompt_tokens}")
print(f"Completion Tokens : {cb.completion_tokens}")
print(f"Total Cost : {cb.total_cost}")


Total Tokens : 1782
Prompt Tokens : 1369
Completion Tokens : 413
Total Cost : 0.0028795


In [22]:
print(response)

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive system

In [23]:
quiz = response.get("quiz")

In [24]:
quiz = json.loads(quiz)

In [25]:
quiz_table = []
for key ,value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}:{option_value}"
            for option , option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table.append({"MCQ":mcq , "Choices":options , "Correct":correct})

In [26]:
quiz_table

[{'MCQ': 'Who coined the term machine learning?',
  'Choices': 'a:Donald Hebb | b:Tom M. Mitchell | c:Arthur Samuel | d:Walter Pitts',
  'Correct': 'c'},
 {'MCQ': "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
  'Choices': 'a:Cyberdyne | b:Cybertron | c:Cybernet | d:Cyberware',
  'Correct': 'b'},
 {'MCQ': 'According to Tom M. Mitchell, what is the definition of machine learning algorithms?',
  'Choices': 'a:A computer program learns from experience and improves performance | b:A computer program mimics human cognitive processes | c:A computer program recognizes patterns using neural networks | d:A computer program predicts future outcomes based on data',
  'Correct': 'a'},
 {'MCQ': 'What are the two main objectives of modern-day machine learning?',
  'Choices': 'a:To develop new computer programs and algorithms | b:To analyze human cognitive processes | c:To classify data and make predictions based on models | d:To study hi

In [27]:
final_quiz = pd.DataFrame(quiz_table)

In [28]:
final_quiz.to_csv("machine_learning.csv",index = False)